### > Setup ffmpeg

In [3]:
%%sh
sudo apt-get update && sudo apt upgrade -y

sudo apt-get install ffmpeg -y

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1796 kB]
Fetched 2025 kB in 2s (1056 kB/s)
Reading package lists...


Reading package lists...
Building dependency tree...
Reading state information...
Calculating upgrade...
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


### > Initialize parameters

In [2]:
import boto3
import sagemaker
from sagemaker.utils import name_from_base
import shutil
import os

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
region = sagemaker_session._region_name

prefix = "multi-modal-search"

s3_client = boto3.client('s3')

video_dir = 'videos' # BigBuckBunny.mp4' #TearsOfSteel.mp4'  #

# load the host and index_name for opensearch
%store -r host
%store -r index_name

print(f"Opensearch hosting url: {host}")
print(f"Opensearch index name: {index_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Opensearch hosting url: qvbu68t99vmrupahu4i2.us-west-2.aoss.amazonaws.com
Opensearch index name: mm-search-2024-03-03-02-39-43-750


### > download and setup videos

In [3]:
if os.path.exists(video_dir):
    # Remove existing folder
    shutil.rmtree(video_dir)
    
# Create new folder
os.makedirs(video_dir)

In [4]:
!cd {video_dir} && curl https://gist.githubusercontent.com/jsturgis/3b19447b304616f18657/raw/a8c1f60074542d28fa8da4fe58c3788610803a65/gistfile1.txt | grep -o 'http[^"]*.mp4' | xargs -n 1 curl -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7692  100  7692    0     0  33669      0 --:--:-- --:--:-- --:--:-- 33736
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  150M  100  150M    0     0   128M      0  0:00:01  0:00:01 --:--:--  128M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  161M  100  161M    0     0  35.1M      0  0:00:04  0:00:04 --:--:-- 35.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2439k  100 2439k    0     0  23.8M      0 --:--:-- --:--:-- --:--:-- 24.0M
  % Total    % Received % Xferd  Average Speed   Tim

In [5]:
CDN_URL = "https://d2yqlwoly7fl0b.cloudfront.net/super-slomo"

SAMPLE_VIDEO = "westiepoo.mov"
!wget -L {CDN_URL}/samples/{SAMPLE_VIDEO} -O videos/{SAMPLE_VIDEO} --no-check-certificate

--2024-03-03 02:55:24--  https://d2yqlwoly7fl0b.cloudfront.net/super-slomo/samples/westiepoo.mov
Resolving d2yqlwoly7fl0b.cloudfront.net (d2yqlwoly7fl0b.cloudfront.net)... 18.161.3.70, 18.161.3.85, 18.161.3.178, ...
Connecting to d2yqlwoly7fl0b.cloudfront.net (d2yqlwoly7fl0b.cloudfront.net)|18.161.3.70|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 8908629 (8.5M) [video/quicktime]
Saving to: ‘videos/westiepoo.mov’

videos/westiepoo.mo 100%[===================>]   8.50M  --.-KB/s    in 0.05s   

2024-03-03 02:55:25 (160 MB/s) - ‘videos/westiepoo.mov’ saved [8908629/8908629]



### > Check videos are available

In [3]:
try:
    entries = os.scandir(video_dir)
except FileNotFoundError as e:
    print(f"Could not scan folder: {e}")
    raise
except OSError as e: 
    print(f"Could not scan folder: {e}")
    raise

### > Process videos

The processing include following steps:
1. extract key frames from video
2. upload to s3 and build the index
3. ingest index to opensearch

In [4]:
from helper import extract_key_frames, upload_frames
from opensearch_util import bulk_index_ingestion
from tqdm import tqdm

for entry in tqdm(entries):
    try:
        if entry.is_file() and entry.path.endswith((".mp4", ".mkv", ".mov")): 

            print(f"process file: {entry.path}...")
            # extract key frames from video
            output_dir, fps = extract_key_frames(entry.path)
            print(f"frame rate is {fps} per second")

            # upload fames to s3
            frames = upload_frames(output_dir, bucket, prefix, entry.path.split('/')[-1], fps)
            print(f"upload frames to s3 bucket...")

            # ingest the index file into opensearch
            sucess, failed = bulk_index_ingestion(host, index_name, frames)
            
    except OSError as e: 
        print(f"Could not read entry {entry.path}: {e}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


0it [00:00, ?it/s]

process file: videos/westiepoo.mov...
frame rate is 25.0 per second
upload frames to s3 bucket...


1it [00:01,  1.06s/it]

Indexed 1 documents
process file: videos/BigBuckBunny.mp4...
frame rate is 24.0 per second
upload frames to s3 bucket...


...

Indexed 316 documents
process file: videos/WeAreGoingOnBullrun.mp4...
frame rate is 24.0 per second
upload frames to s3 bucket...


14it [06:21, 35.35s/it]

Indexed 49 documents
process file: videos/WhatCarCanYouGetForAGrand.mp4...
frame rate is 29.97002997002997 per second
upload frames to s3 bucket...


15it [07:09, 28.65s/it]

Indexed 295 documents


### > Test Multi-modal text search

In [8]:
from helper import embed_and_search, render_search_result
from IPython.display import display, clear_output

results = embed_and_search(text_query="Game of thrones", 
                           host = host,
                           index_name=index_name, 
                          numb = 3)

display(render_search_result(results))

### > Test MM mage Search

In [11]:
results = embed_and_search(image_path="test02.jpeg", 
                           host = host,
                           index_name=index_name, 
                          numb = 3)

display(render_search_result(results))

### > Interactive UI

Have fun.....!

Here are some sample prompts:

- cute dog running on grass
- flying squiral
- a bunny flying a kite
- Game of thrones

In [14]:
import ipywidgets as ipw
class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa):
        self.qa = qa
        self.name = None
        self.b=None
        self.out = ipw.Output()
        self.session_id = None

    def start_chat(self):
        print("Let's chat!")
        display(self.out)
        self.chat(None)

    def chat(self, _):
        if self.name is None:
            prompt = ""
        else:
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value=f"Thinking...")
                display(thinking)
                try:
                    results = embed_and_search(text_query=prompt, 
                           host = host,
                           index_name=index_name, 
                          numb = 3)
                    output_results = render_search_result(results)
                except Exception as e:
                    print(e)
                    output_results = "No answer"

                thinking.value=""
                print("AI:")
                display(output_results)
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You: ", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

In [15]:
agent_runtime_client = boto3.client('bedrock-runtime')

chat = ChatUX(agent_runtime_client)
chat.start_chat()

Let's chat!


Output()